# OPTUNA IN MACHINE LEARNING

#### Terms in Optuna

1. Study: A Study in Optuna is an optimization session that encompasses multiple trails. Essential objective is to optimize the objective fnc; i.e. the training and  finding the best hyperparameters are called as study in term's of Optuna's language.
2. Trial: A trial is a single iteration of the optimization process where a specific set of hyperparameters is evaluated. Each trial runs the obj func once with a distinct set of hyperparameters.
3. Trial Parameters: Specific hyperparameters values chosen during a trial. Each trial will have a unique combination of hyperparameters that are evaluated to see how they impact the objective function. E.g. - Different different learning rates, batch_sizes during each trial.
4. Objective Function: The fnc to be optimized(minimized or maximized) during the hyperparameter search. It takes hyperparameters as input and return a value (eg accuracy, loss, etc) that Optuna tries to optimize. E.g. - In a classification tack, the obj fnc could be the cross-entropy loss which Optuna seeks to optimize.
5. Sampler: A sampler is the algorithm that suggests which hyperparameters should be evaluated next. Optuna uses the Tree-structured Parzen Estimator(TPE) by default(TPE uses Bayesian Optimization), but can also support other sampling like Random Sampling or even Custom Sampling,

In [11]:
# !uv pip install optuna

In [1]:
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

df = pd.read_csv(url, names=columns)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
import numpy as np

# Missing Value Imputation
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0,np.nan)

df.fillna(df.mean(), inplace=True)
print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [3]:
X = df.drop('Outcome', axis=1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

# Optional Scaling
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')

Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
# Now we will define an objective function for the workflow of optuna

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
# Define the objective fnc

def objective(trial):
    # Suggest values of the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    # Here the search space will be in the Range [50,200] for n_estimators
    # and range of [3,20] for max_depth

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )
    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
# As we are trying to 'Maximize' our accuracy so our direction of the objective function will be to maximize it
study.optimize(objective, n_trials=100) # Running 50 trials to find the optimal hyperparameters

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best trial Hyperparameters: {study.best_trial.params}')

from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

test_accuracy = accuracy_score(y_test, y_pred)

print(f'Test accuracy ith best hyperparameters: {test_accuracy:.2f}')

[I 2025-07-02 18:09:34,177] A new study created in memory with name: no-name-ff348ff6-4448-4c70-8572-50bcfebb1d4f
[I 2025-07-02 18:09:34,487] Trial 0 finished with value: 0.7783985102420856 and parameters: {'n_estimators': 106, 'max_depth': 18}. Best is trial 0 with value: 0.7783985102420856.
[I 2025-07-02 18:09:34,709] Trial 1 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 59, 'max_depth': 13}. Best is trial 0 with value: 0.7783985102420856.
[I 2025-07-02 18:09:35,257] Trial 2 finished with value: 0.756052141527002 and parameters: {'n_estimators': 173, 'max_depth': 4}. Best is trial 0 with value: 0.7783985102420856.
[I 2025-07-02 18:09:35,666] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 130, 'max_depth': 11}. Best is trial 0 with value: 0.7783985102420856.
[I 2025-07-02 18:09:36,264] Trial 4 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 200, 'max_depth': 10}. Best is trial 0 with value: 0.7783985

Best trial accuracy: 0.7839851024208566
Best trial Hyperparameters: {'n_estimators': 73, 'max_depth': 18}
Test accuracy ith best hyperparameters: 0.76


In [14]:
# Now we will use RandomSampler

# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())
# As we are trying to 'Maximize' our accuracy so our direction of the objective function will be to maximize it
study.optimize(objective, n_trials=100) # Running 50 trials to find the optimal hyperparameters

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best trial Hyperparameters: {study.best_trial.params}')

from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

test_accuracy = accuracy_score(y_test, y_pred)

print(f'Test accuracy ith best hyperparameters: {test_accuracy:.2f}')

[I 2025-07-02 18:01:46,900] A new study created in memory with name: no-name-0a892328-555f-46ba-a887-a6036f217a80
[I 2025-07-02 18:01:47,366] Trial 0 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 109, 'max_depth': 5}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-07-02 18:01:47,665] Trial 1 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 72, 'max_depth': 5}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-07-02 18:01:48,205] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 116, 'max_depth': 8}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-07-02 18:01:48,474] Trial 3 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 58, 'max_depth': 8}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-07-02 18:01:48,933] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 98, 'max_depth': 14}. Best is trial 1 with value: 0.77467411545

Best trial accuracy: 0.7858472998137803
Best trial Hyperparameters: {'n_estimators': 121, 'max_depth': 15}
Test accuracy ith best hyperparameters: 0.76


In [15]:
# Now we will use Grid Search

# We have to explicitly mention our search space first

search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
# As we are trying to 'Maximize' our accuracy so our direction of the objective function will be to maximize it
study.optimize(objective, n_trials=100) # Running 50 trials to find the optimal hyperparameters

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best trial Hyperparameters: {study.best_trial.params}')

from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

test_accuracy = accuracy_score(y_test, y_pred)

print(f'Test accuracy ith best hyperparameters: {test_accuracy:.2f}')

[I 2025-07-02 18:04:47,419] A new study created in memory with name: no-name-f2b3e42d-d08b-4678-97e2-377437c3e93d
[I 2025-07-02 18:04:47,731] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-07-02 18:04:48,290] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-07-02 18:04:48,497] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-07-02 18:04:48,832] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-07-02 18:04:49,232] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

Best trial accuracy: 0.7746741154562384
Best trial Hyperparameters: {'n_estimators': 50, 'max_depth': 5}
Test accuracy ith best hyperparameters: 0.74


#### Optuna Visualization

In [7]:
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [18]:
# !uv pip install matplotlib plotly

Using Python 3.10.12 environment at: PyTorch
Resolved 13 packages in 183ms                                        
Prepared 2 packages in 2.16s                                             
Installed 2 packages in 25ms                                
 + narwhals==1.45.0
 + plotly==6.2.0


In [8]:
# 1. Optimization history - trial number vs accuracy graph

plot_optimization_history(study).show()

In [9]:
# 2. Parallel Coordinate Plot - SHows the hotspots

plot_parallel_coordinate(study).show()

In [10]:
# 3. Slice Plot - Individually plots objective function vs each hyperparameter

plot_slice(study).show()

In [11]:
# Contour plot - Dense colour shows the best possible location of the objective fnc
plot_contour(study).show()

In [12]:
# Importance plot - gives importance of each hyperparameter

plot_param_importances(study).show()

##### Define by run is used in Optuna to search from Dynamic Search Spaces. We can consider the models/ML algorithms as hyperparameter which gives the flexibility to know which ML model is the best for our problem and also gives the best hyperparameters for that particular algorithm. - This challenge is solved by Optuna!!! Every algorithm will have different/dynamic search space wrt the algorith it is using. Optuna also supports distributed computing to make the computing faster.

In [14]:
# Optimizing multiple ML models

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import  SVC

In [15]:
def objective(trial):
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])
    if classifier_name=='SVM':
        # SVM hyperparams
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)
    elif classifier_name=='RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])
        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )
    elif classifier_name == 'GradientBoosting':
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-val and return mean of acu
    score = cross_val_score(model, X_train, y_train, cv=3,scoring='accuracy').mean()
    return score

In [17]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best trial Hyperparameters: {study.best_trial.params}')

[I 2025-07-02 18:57:23,974] A new study created in memory with name: no-name-b89aef68-5b6f-4a7e-85cd-e4d3999cc84f
[I 2025-07-02 18:57:24,689] Trial 0 finished with value: 0.7653631284916201 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 258, 'learning_rate': 0.011472826268146236, 'max_depth': 3, 'min_samples_split': 9, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.7653631284916201.
[I 2025-07-02 18:57:25,267] Trial 1 finished with value: 0.7709497206703911 and parameters: {'classifier': 'RandomForest', 'n_estimators': 244, 'max_depth': 20, 'min_samples_split': 4, 'min_samples_leaf': 7, 'bootstrap': False}. Best is trial 1 with value: 0.7709497206703911.
[I 2025-07-02 18:57:25,317] Trial 2 finished with value: 0.7858472998137801 and parameters: {'classifier': 'SVM', 'C': 18.041504378844213, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 2 with value: 0.7858472998137801.
[I 2025-07-02 18:57:25,555] Trial 3 finished with value: 0.756052141527002 and parame

Best trial accuracy: 0.7895716945996275
Best trial Hyperparameters: {'classifier': 'SVM', 'C': 0.13163973212477242, 'kernel': 'linear', 'gamma': 'auto'}


In [19]:
from sklearn.metrics import accuracy_score

params = study.best_trial.params
model_class = params['classifier']  # e.g., 'SVM'

# Remove 'classifier' and pass the rest as kwargs
model_kwargs = {k: v for k, v in params.items() if k != 'classifier'}
model_kwargs['random_state'] = 42

# You would typically map string name to actual class
classifier_map = {
    'SVM': SVC,
    # Add other mappings if needed
}

# Train using the best hyperparameters from Optuna
best_model = classifier_map[model_class](**model_kwargs)

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

test_accuracy = accuracy_score(y_test, y_pred)

print(f'Test accuracy ith best hyperparameters: {test_accuracy:.2f}')

Test accuracy ith best hyperparameters: 0.74


In [20]:
# We can also create dataframe of the study

study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.765363,2025-07-02 18:57:23.975780,2025-07-02 18:57:24.689564,0 days 00:00:00.713784,NaN,NaN,GradientBoosting,NaN,NaN,0.011473,3.0,5.0,9.0,258.0,COMPLETE
1,1,0.770950,2025-07-02 18:57:24.690158,2025-07-02 18:57:25.267910,0 days 00:00:00.577752,NaN,False,RandomForest,NaN,NaN,NaN,20.0,7.0,4.0,244.0,COMPLETE
2,2,0.785847,2025-07-02 18:57:25.268676,2025-07-02 18:57:25.317121,0 days 00:00:00.048445,18.041504,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.756052,2025-07-02 18:57:25.317748,2025-07-02 18:57:25.555334,0 days 00:00:00.237586,NaN,True,RandomForest,NaN,NaN,NaN,5.0,5.0,4.0,87.0,COMPLETE
4,4,0.757914,2025-07-02 18:57:25.555879,2025-07-02 18:57:25.955644,0 days 00:00:00.399765,NaN,True,RandomForest,NaN,NaN,NaN,6.0,6.0,2.0,142.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.746741,2025-07-02 18:57:45.613228,2025-07-02 18:57:45.639297,0 days 00:00:00.026069,0.147560,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.789572,2025-07-02 18:57:45.639946,2025-07-02 18:57:45.661181,0 days 00:00:00.021235,0.139860,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.713222,2025-07-02 18:57:45.661782,2025-07-02 18:57:45.684368,0 days 00:00:00.022586,0.142585,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.756052,2025-07-02 18:57:45.685024,2025-07-02 18:57:46.471904,0 days 00:00:00.786880,NaN,NaN,GradientBoosting,NaN,NaN,0.122128,5.0,4.0,8.0,147.0,COMPLETE


In [21]:
study.trials_dataframe()['params_classifier'].value_counts()

params_classifier
SVM                 78
RandomForest        12
GradientBoosting    10
Name: count, dtype: int64

In [22]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.744134
RandomForest        0.761794
SVM                 0.772120
Name: value, dtype: float64

In [1]:
import sys

print(sys.executable)

/home/ml02/Downloads/PyTorch/PyTorch/bin/python


# OPTUNA IN DEEP LEARNING

#### How to train a neural network model in optuna

Optuna creates a study object which contains trials. Each trial is sent to the objective function. Objective function is used to define - Search Space, Model Init, Param Init, Training loop, Evaluation Loop which returns the objective metric.

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [3]:
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device:{device}")

Using device:cuda


In [4]:
df = pd.read_csv("/home/ml02/Downloads/PyTorch/archive/fashion-mnist_train.csv")
df.shape

(60000, 785)

In [5]:
X = df.iloc[:,1:].values
y = df.iloc[:, 0].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
X_train = X_train/255.0
X_test = X_test/255.0

In [8]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    def __len__(self):
        return len(self.features)
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [9]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [10]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [11]:
len(train_loader)

1500

In [12]:
#NB - The below architecture has as drawback - We are using same neurons for every layer
class MyNN(nn.Module):
    def __init__(self, input_dim, output_dim, num_hidden_layers, neurons_per_layer):
        super().__init__()
        layers = [] # This will store the layers dynamically in every trial
        for i in range(num_hidden_layers):
            layers.append(nn.Linear(input_dim, neurons_per_layer))
            layers.append(nn.BatchNorm1d(neurons_per_layer))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.3))
            input_dim = neurons_per_layer
        layers.append(nn.Linear(neurons_per_layer, output_dim))
        self.model = nn.Sequential(*layers) # Unpacking the array here as sequential takes input one by one

    def forward(self, x):
        return self.model(x)

In [13]:
# Now we will make our example objective function
# and will only focus on two things -
## 1. Number of hidden layers
## 2. Number of neurons per layer

def objective(trial):
    # next hyperparameter values from the search space
    num_hidden_layers = trial.suggest_int("num_hidden_layers", 1,5)
    neurons_per_layer = trial.suggest_int("neurons_per_layer", 8, 128, step=8)

    # model init
    input_dim = 784
    output_dim = 10

    model = MyNN(input_dim, output_dim, num_hidden_layers, neurons_per_layer)
    model.to(device)

    # param init
    lr = 0.01
    epochs = 100

    # optimizer selection
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

    # training loop - We will not print values of loss here
    model.train()
    for epoch in range(epochs):
        for batch_features, batch_labels in train_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            outputs = model(batch_features) # Forward pass
            loss = criterion(outputs, batch_labels)
            # back pass
            optimizer.zero_grad()
            loss.backward()
            # update grads
            optimizer.step()

    # evaluation
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for batch_features, batch_labels in test_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            outputs = model(batch_features)
            _, predicted = torch.max(outputs, 1)
            total += batch_labels.shape[0]
            correct = correct + (predicted == batch_labels).sum().item()
        accuracy = correct/total
    return accuracy

In [14]:
# Now we will create an Optuna study
import optuna

study = optuna.create_study(direction='maximize')

[I 2025-07-07 19:23:33,061] A new study created in memory with name: no-name-51296a41-b765-4c0e-86c1-31c67aedc18c


In [15]:
study.optimize(objective, n_trials=10)

[I 2025-07-07 19:26:56,750] Trial 0 finished with value: 0.7855833333333333 and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 8}. Best is trial 0 with value: 0.7855833333333333.
[I 2025-07-07 19:31:44,820] Trial 1 finished with value: 0.8934166666666666 and parameters: {'num_hidden_layers': 4, 'neurons_per_layer': 112}. Best is trial 1 with value: 0.8934166666666666.
[I 2025-07-07 19:36:37,260] Trial 2 finished with value: 0.8946666666666667 and parameters: {'num_hidden_layers': 4, 'neurons_per_layer': 112}. Best is trial 2 with value: 0.8946666666666667.
[I 2025-07-07 19:42:21,163] Trial 3 finished with value: 0.8860833333333333 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 120}. Best is trial 2 with value: 0.8946666666666667.
[I 2025-07-07 19:46:21,853] Trial 4 finished with value: 0.8656666666666667 and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 32}. Best is trial 2 with value: 0.8946666666666667.
[I 2025-07-07 19:49:22,589] Trial 5 finishe

In [16]:
study.best_value

0.8946666666666667

In [17]:
study.best_params

{'num_hidden_layers': 4, 'neurons_per_layer': 112}

#### Now we will continue to update the objective function and will add few more params in hyperparam searching

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device:{device}")

Using device:cuda


In [3]:
df = pd.read_csv("/home/ml02/Downloads/PyTorch/archive/fashion-mnist_train.csv")
df.shape

(60000, 785)

In [4]:
X = df.iloc[:,1:].values
y = df.iloc[:, 0].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
X_train = X_train/255.0
X_test = X_test/255.0

In [7]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    def __len__(self):
        return len(self.features)
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [8]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [9]:
#NB - The below architecture has as drawback - We are using same neurons for every layer
class MyNN(nn.Module):
    def __init__(self, input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout):
        super().__init__()
        layers = [] # This will store the layers dynamically in every trial
        for i in range(num_hidden_layers):
            layers.append(nn.Linear(input_dim, neurons_per_layer))
            layers.append(nn.BatchNorm1d(neurons_per_layer))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            input_dim = neurons_per_layer
        layers.append(nn.Linear(neurons_per_layer, output_dim))
        self.model = nn.Sequential(*layers) # Unpacking the array here as sequential takes input one by one

    def forward(self, x):
        return self.model(x)

In [10]:
# Now we will make our example objective function
# and will only focus on two things -
## 1. Number of hidden layers
## 2. Number of neurons per layer

def objective(trial):
    # next hyperparameter values from the search space
    num_hidden_layers = trial.suggest_int("num_hidden_layers", 1,5)
    neurons_per_layer = trial.suggest_int("neurons_per_layer", 8, 128, step=8)
    epochs = trial.suggest_int("epochs", 10, 50, step=10)
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True) # The values will be in logarithmic scale
    dropout_rate = trial.suggest_float("dropout", 0.1, 0.5, step=0.1)
    batch_size = trial.suggest_categorical("batch_size", [16,32,64,128])
    optimizer_name = trial.suggest_categorical("optimizer", ['Adam', 'SGD', 'RMSprop'])
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)

    # model init
    input_dim = 784
    output_dim = 10
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
    model = MyNN(input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate)
    model.to(device)

    # optimizer selection
    criterion = nn.CrossEntropyLoss()
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)

    # training loop - We will not print values of loss here
    model.train()
    for epoch in range(epochs):
        for batch_features, batch_labels in train_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            outputs = model(batch_features) # Forward pass
            loss = criterion(outputs, batch_labels)
            # back pass
            optimizer.zero_grad()
            loss.backward()
            # update grads
            optimizer.step()

    # evaluation
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for batch_features, batch_labels in test_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            outputs = model(batch_features)
            _, predicted = torch.max(outputs, 1)
            total += batch_labels.shape[0]
            correct = correct + (predicted == batch_labels).sum().item()
        accuracy = correct/total
    return accuracy

In [11]:
import optuna

study = optuna.create_study(direction='maximize')

[I 2025-07-07 20:42:02,607] A new study created in memory with name: no-name-01d1e774-3b13-426e-a4e6-6cef166a8263


In [12]:
study.optimize(objective, n_trials=10)

[I 2025-07-07 20:42:29,247] Trial 0 finished with value: 0.8781666666666667 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 112, 'epochs': 10, 'lr': 0.00023473466577846877, 'dropout': 0.30000000000000004, 'batch_size': 128, 'optimizer': 'RMSprop', 'weight_decay': 3.8709483389315404e-05}. Best is trial 0 with value: 0.8781666666666667.
[I 2025-07-07 20:43:19,660] Trial 1 finished with value: 0.8655833333333334 and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 32, 'epochs': 30, 'lr': 0.0006317252980147981, 'dropout': 0.30000000000000004, 'batch_size': 128, 'optimizer': 'RMSprop', 'weight_decay': 5.373048924186291e-05}. Best is trial 0 with value: 0.8781666666666667.
[I 2025-07-07 20:44:20,216] Trial 2 finished with value: 0.8813333333333333 and parameters: {'num_hidden_layers': 4, 'neurons_per_layer': 112, 'epochs': 30, 'lr': 0.00013226849854707735, 'dropout': 0.2, 'batch_size': 128, 'optimizer': 'RMSprop', 'weight_decay': 0.0001327606301060182}. Best is trial 2

In [13]:
study.best_value

0.8813333333333333

In [14]:
study.best_params

{'num_hidden_layers': 4,
 'neurons_per_layer': 112,
 'epochs': 30,
 'lr': 0.00013226849854707735,
 'dropout': 0.2,
 'batch_size': 128,
 'optimizer': 'RMSprop',
 'weight_decay': 0.0001327606301060182}

# Integrating MlFlow With Optuna to track trials

In [ ]:
# # !uv pip install mlflow

In [10]:
# !uv pip install optuna-integration

Using Python 3.10.12 environment at: PyTorch
Resolved 14 packages in 1.55s                                        
Prepared 1 package in 180ms                                              
Installed 1 package in 3ms=4.4.0                            
 + optuna-integration==4.4.0


In [1]:
import torch
import pandas as pd
import numpy as np
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
df = pd.read_csv("/home/ml02/Downloads/PyTorch/archive/fashion-mnist_train.csv")
X = df.iloc[:,1:].values
y = df.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train/255.0
X_test = X_test/255.0

In [3]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    def __len__(self):
        return len(self.features)
    def __getitem__(self, item):
        return self.features[item], self.labels[item]

train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [4]:
class MyNN(nn.Module):
    def __init__(self, input_dim, output_dim, num_hiddden_layers, neurons_per_layer, dropout):
        super().__init__()
        layers = []
        for i in range(num_hiddden_layers):
            layers.append(nn.Linear(input_dim, neurons_per_layer))
            layers.append(nn.BatchNorm1d(neurons_per_layer))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            input_dim = neurons_per_layer
        layers.append(nn.Linear(neurons_per_layer, output_dim))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

 Don’t need to manually write with mlflow.start_run(): when using Optuna with MLflowCallback. It’s handled automatically. MLflowCallback from Optuna ,wraps each trial in an MLflow run behind the scenes. So,
 ```
 with mlflow.start_run():
    ...
```
is automatically done by the callback during each trial.

You just need to:

> Create your MLflowCallback;
> Decorate your objective function with @mlflc.track_in_mlflow();
> Run the study — and everything gets logged!

In [5]:
import optuna
import mlflow
from optuna.integration.mlflow import MLflowCallback

# mlflow.set_tracking_uri("http://localhost:5000")
# mlflow.set_experiment("Optuna Hyperparameter Search 1")

mlflc = MLflowCallback(
tracking_uri="http://localhost:5000",
metric_name="Accuracy"
)

@mlflc.track_in_mlflow()
def objective(trial):
    num_hidden_layers = trial.suggest_int("num_hidden_layers", 1,5)
    neurons_per_layer = trial.suggest_int("neurons_per_layer", 8, 128, step=8)
    epochs = trial.suggest_int("epochs", 10, 100, step=10)
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    dropout_rate = trial.suggest_float("dropout", 0.1, 0.5, step=0.1)
    batch_size = trial.suggest_categorical("batch_size", [16,32,64,128])
    # Optional: Add label smoothing as a hyperparameter
    label_smoothing = trial.suggest_float("label_smoothing", 0.0, 0.2, step=0.05)
    optimizer_name = trial.suggest_categorical("optimizer", ['Adam', 'SGD', 'RMSprop', 'Rprop', 'LBFGS'])
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)

    # model init
    input_dim = 784
    output_dim = 10
    train_loader = DataLoader(train_dataset, batch_size, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size, shuffle=False, pin_memory=True)
    model = MyNN(input_dim, output_dim, num_hidden_layers, neurons_per_layer, dropout_rate)
    model.to(device)

    # optimizer selection

    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'Rprop':
        optimizer = optim.Rprop(model.parameters(), lr=lr, etas=(0.5, 1.2), step_sizes=(1e-10, 100))
    elif optimizer_name == 'LBFGS':
        optimizer = optim.LBFGS(model.parameters(), lr=lr, max_iter=15, tolerance_grad=1e-10, tolerance_change=1e-12)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)

    # training loop
    model.train()
    for epoch in range(epochs):
        for batch_features, batch_labels in train_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            if optimizer_name == 'LBFGS':
                def closure():
                    optimizer.zero_grad()
                    outputs = model(batch_features)
                    loss = criterion(outputs, batch_labels)
                    loss.backward()
                    return loss
                optimizer.step(closure)
            else:
                outputs = model(batch_features)
                loss = criterion(outputs, batch_labels)
                # back pass
                optimizer.zero_grad()
                loss.backward()
                # update grads
                optimizer.step()

    # evaluatiom
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for batch_features, batch_labels in test_loader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            outputs = model(batch_features)
            _, predicted = torch.max(outputs, 1)
            total += batch_features.shape[0]
            correct += (predicted == batch_labels).sum().item()
        accuracy = correct/total
    return accuracy

/tmp/ipykernel_166386/3442898715.py:8: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(
/tmp/ipykernel_166386/3442898715.py:13: ExperimentalWarning: track_in_mlflow is experimental (supported from v2.9.0). The interface can change in the future.
  @mlflc.track_in_mlflow()


In [6]:
study = optuna.create_study(study_name="First_Optuna_Hyperparameter_Search", direction="maximize")

[I 2025-07-08 16:43:02,642] A new study created in memory with name: First_Optuna_Hyperparameter_Search


In [7]:
study.optimize(objective, n_trials=50, callbacks=[mlflc])

2025/07/08 16:43:04 INFO mlflow.tracking.fluent: Experiment with name 'First_Optuna_Hyperparameter_Search' does not exist. Creating a new experiment.
[I 2025-07-08 16:43:35,218] Trial 0 finished with value: 0.8689166666666667 and parameters: {'num_hidden_layers': 4, 'neurons_per_layer': 32, 'epochs': 20, 'lr': 0.00029765816732581007, 'dropout': 0.2, 'batch_size': 128, 'label_smoothing': 0.0, 'optimizer': 'Adam', 'weight_decay': 0.00015518267720959756}. Best is trial 0 with value: 0.8689166666666667.


🏃 View run 0 at: http://localhost:5000/#/experiments/605049891641191207/runs/9fe98f09c0294489886a9688412e9abe
🧪 View experiment at: http://localhost:5000/#/experiments/605049891641191207
🏃 View run 0 at: http://localhost:5000/#/experiments/605049891641191207/runs/9fe98f09c0294489886a9688412e9abe
🧪 View experiment at: http://localhost:5000/#/experiments/605049891641191207


[I 2025-07-08 16:46:54,570] Trial 1 finished with value: 0.54875 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 16, 'epochs': 60, 'lr': 2.2137897447501633e-05, 'dropout': 0.2, 'batch_size': 32, 'label_smoothing': 0.1, 'optimizer': 'SGD', 'weight_decay': 0.0003129368585252753}. Best is trial 0 with value: 0.8689166666666667.


🏃 View run 1 at: http://localhost:5000/#/experiments/605049891641191207/runs/9fe0ca6b018542cc9f7b1d2acfdd006b
🧪 View experiment at: http://localhost:5000/#/experiments/605049891641191207
🏃 View run 1 at: http://localhost:5000/#/experiments/605049891641191207/runs/9fe0ca6b018542cc9f7b1d2acfdd006b
🧪 View experiment at: http://localhost:5000/#/experiments/605049891641191207


[I 2025-07-08 16:52:11,478] Trial 2 finished with value: 0.293 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 112, 'epochs': 60, 'lr': 0.00011238180034710855, 'dropout': 0.30000000000000004, 'batch_size': 64, 'label_smoothing': 0.15000000000000002, 'optimizer': 'Rprop', 'weight_decay': 0.00041725843676110476}. Best is trial 0 with value: 0.8689166666666667.


...

🏃 View run 47 at: http://localhost:5000/#/experiments/605049891641191207/runs/54a80e5e9b7347bd8f672cac36ff7ee1
🧪 View experiment at: http://localhost:5000/#/experiments/605049891641191207
🏃 View run 47 at: http://localhost:5000/#/experiments/605049891641191207/runs/54a80e5e9b7347bd8f672cac36ff7ee1
🧪 View experiment at: http://localhost:5000/#/experiments/605049891641191207


[I 2025-07-08 20:40:12,774] Trial 48 finished with value: 0.70375 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 120, 'epochs': 40, 'lr': 2.5281333333923215e-05, 'dropout': 0.2, 'batch_size': 32, 'label_smoothing': 0.15000000000000002, 'optimizer': 'SGD', 'weight_decay': 1.001254052520279e-05}. Best is trial 42 with value: 0.8971666666666667.


🏃 View run 48 at: http://localhost:5000/#/experiments/605049891641191207/runs/0b9be09607304427b2885ee598834e97
🧪 View experiment at: http://localhost:5000/#/experiments/605049891641191207
🏃 View run 48 at: http://localhost:5000/#/experiments/605049891641191207/runs/0b9be09607304427b2885ee598834e97
🧪 View experiment at: http://localhost:5000/#/experiments/605049891641191207


[I 2025-07-08 20:49:22,340] Trial 49 finished with value: 0.08033333333333334 and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 24, 'epochs': 60, 'lr': 0.00016214771607804684, 'dropout': 0.1, 'batch_size': 32, 'label_smoothing': 0.1, 'optimizer': 'LBFGS', 'weight_decay': 1.2878101760084487e-05}. Best is trial 42 with value: 0.8971666666666667.


🏃 View run 49 at: http://localhost:5000/#/experiments/605049891641191207/runs/10b8b39b657f4a09ab683bce2add4c98
🧪 View experiment at: http://localhost:5000/#/experiments/605049891641191207
🏃 View run 49 at: http://localhost:5000/#/experiments/605049891641191207/runs/10b8b39b657f4a09ab683bce2add4c98
🧪 View experiment at: http://localhost:5000/#/experiments/605049891641191207


In [8]:
study.best_value

0.8971666666666667

In [9]:
study.best_params

{'num_hidden_layers': 5,
 'neurons_per_layer': 120,
 'epochs': 40,
 'lr': 0.00014945226423368488,
 'dropout': 0.2,
 'batch_size': 32,
 'label_smoothing': 0.1,
 'optimizer': 'RMSprop',
 'weight_decay': 1.5509457304206577e-05}